In [47]:
#outputs all variables on their own lines in the OUT console

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from bs4 import BeautifulSoup as bs
import os
import urllib3 
import subprocess
import requests

#Pre-soup methods (URL request -> page contents -> text)

def getPageText(selector):
    try: 
        sourcepages.get(selector)
        response=requests.get(sourcepages.get(selector))
        PAGE=response.text
        #print (len(PAGE))
        return PAGE, len(PAGE)
    except ValueError:
        #print (len(PAGE))   #MissingSchema exception
        print ("getPageText: Not an appropriate page identifier: check sourcepages dict")
        return 0


def getPageTextTest(selector):
    try:
        assert getPageText(selector) != 0
    except AssertionError:
        print("getPageTextTest: Entered an invalid page identifier: ", selector)
    
#a test suite for the above
def testPageGetting(selector):
    getPageTextTest(selector)
    #getPageText(selector)  #test that this fails properly - and it does
    

class specialSoup(bs):
    tagtarget="li"
    
    def findAllTargets(self, tagtarget):
        """gets litargs array"""
        targettags=self.find_all(tagtarget)
        return targettags
    
def orderRecords(recsarray):
    """get your item records in a nice orderly list of dictionaries"""
    recordlist=[]
    for record in recsarray:
        recordcontents={}
        innertags=[]
        recordcontents["id"]=record["data-spotid"]
        
        for tag in record.descendants:
            innertags.append(tag)
        
        titles=(innertags[12].text).split()
        #print(titles)
                
        recordcontents["href"]=innertags[5].attrs["href"]
        recordcontents["name"]=innertags[8].attrs["alt"]
        try:
            recordcontents["locn"]=titles[1]
            recordcontents["type"]=" ".join(titles[2:])
            #recordcontents["type"]=titles[-1]
        except IndexError:
            recordcontents["locn"]=innertags[27].title().strip()
            recordcontents["type"]=innertags[29].title().strip()
    
        recordlist.append(recordcontents)
    return recordlist   #if this is off by one indent, will overwrite each time and return only the last record value    


def getPOIsfromURL(url):
    urltext=(getPage(url))[0]
    pagesoup=specialSoup(urltext, "html5lib")
    tagtarget="li"
    litags=pagesoup.findAllTargets(tagtarget)
    
    recs=findPOIrecords(litags)
    POIRecs=orderRecords(recs)
    
    return POIRecs

def findPOIrecords(targetsarray):
    """extract POI records; gets recsarray from litargs"""
    recsarray=[]
    for tag in targetsarray:
        if tag.attrs["class"][0]=="c-media__list":
            recsarray.append(tag)
    return recsarray

#DEPRECATED
#put the above two cells into a method to return a next page
def getNextPage(soup):
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                target.append(tag)
        except KeyError:
            pass
    
    nextpage=target[0]["href"]
    return nextpage

#filtered by the --mono-base-arrow to actually get the NEXT
#--mono-base-arrow-l if you want to get the previous page

def getNextPage2(soup):
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                if tag.span["class"][1] == "--mono-base-arrow":
                    target.append(tag)
        except KeyError:
            pass
    
    nextpage=target[0]["href"]
    return nextpage

def getPage(url):
    try:
        response=requests.get(url)
        PAGE=response.text
        #print (len(PAGE))
        return PAGE, len(PAGE)
    except ValueError:
        return "Nope Nope Nope"

def getNextPagefromURL(url):
    urltext=(getPage(url))[0]
    soup=specialSoup(urltext, "html5lib")
    
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                if tag.span["class"][1] == "--mono-base-arrow":
                    target.append(tag)
        except KeyError:
            pass
    
    if len(target)==0:
        #print("No next page")
        nextpage=""
    else:
        nextpage=target[0]["href"]
        
    return nextpage


def getAllPages(url):
    result=url
    resultsindex=[]
    resultsindex.append(url)
    
    while (len(getNextPagefromURL(url))) > 0:
        url=getNextPagefromURL(url)
        resultsindex.append(url)
        
    return resultsindex

In [7]:
#modified body of getNextPage method to actually get the NEXT page
atags=tour2soup.find_all("a")
len(atags)
target=[]
for n in atags:
    try: 
        #print(type(n["class"]))
        #print ("c-pagenarion__navi" in n["class"])
        if ("c-pagenarion__navi") in n["class"]:     #apparently == True, == False does not work
            #print(n)
            print(n.span)
            print(n.span.text)
            print(n.span["class"][1])
            if n.span["class"][1] == "--mono-base-arrow":
                print("THISSSSS")
                target.append(n)
    except KeyError:
        pass

print(target)

In [ ]:
#Project-Specific Settings

sourcepages={
"tour" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max",
"eat" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max",
"shop" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max",
}

selector = "shop"
shoppage = getPageText(selector)
shoppage = shoppage[0]  #bc. tuple is returned with PAGE text (string), and size

SOUP = bs(shoppage, "html5lib")

mySpecialSoup=specialSoup(shoppage, "html5lib")
tagtarg="li"
lis=mySpecialSoup.findAllTargets(tagtarg)
recs=findPOIrecords(lis)
print(len(recs), " records on page\n")

orderedRecs=orderRecords(recs)
for i in orderedRecs: print(i)

In [29]:
#Some test pages
tpage="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-72/?cost=min-max"
epage="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/p-3/?cost=min-max"
teat=sourcepages.get("eat")

st=sourcepages.get("shop")
tt=sourcepages.get("tour")
et=sourcepages.get("eat")

#this method is enough to pull all the ordered POI records off a single page
r=getPOIsfromURL(tpage)
for i in r: print(i)
    
print(getNextPagefromURL(epage))
print(getNextPagefromURL(tpage))
print(getNextPagefromURL(teat))
print(getNextPagefromURL(st))

{'id': 'lj0003568', 'name': '波除稲荷神社', 'href': 'https://livejapan.com/ja/in-tsukiji/spot-lj0003568/', 'locn': '築地', 'type': '神社'}
{'id': 'lj0002566', 'name': '宮下公園', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0002566/', 'locn': '渋谷', 'type': '公園'}
{'id': 'lj0003610', 'name': '神楽坂通り', 'href': 'https://livejapan.com/ja/in-kagurazaka/spot-lj0003610/', 'locn': '神楽坂', 'type': 'その他'}
{'id': 'lj0005612', 'name': '日本橋とやま館 観光交流サロン', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0005612/', 'locn': '観光交流サロン', 'type': '東京駅'}
{'id': 'lj0000014', 'name': 'T-CAT外国人観光案内所', 'href': 'https://livejapan.com/ja/in-ningyocho_monzen-nakacho_kasai/spot-lj0000014/', 'locn': '人形町・門前仲町・葛西', 'type': '観光案内所'}
{'id': 'lj0002406', 'name': '大泉アニメゲート', 'href': 'https://livejapan.com/ja/in-itabashi_nerima/spot-lj0002406/', 'locn': '板橋・練馬', 'type': 'ランドマーク・モニュメント'}
{'id': 'lj0005606', 'name': '都営交通案内所', 'href': 'https://livejapan.com/ja/in-shinjuku/spot-lj0005606/', 'locn': '新宿', 'type': '観光案内所

In [44]:
st
tt
et

#these are the pages we will look through
shop_all=(getAllPages(st))
tt_all=(getAllPages(tt))
et_all=(getAllPages(et))

print(len(shop_all), len(tt_all), len(et_all))

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max'

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max'

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max'

KeyboardInterrupt: 

In [ ]:
et_all[168]

In [48]:
#Our final output arrays
shopPOIlist=[]
eatPOIlist=[]
tourPOIlist=[]

for surl in shop_all:
    for item in getPOIsfromURL(surl):
        if item not in shopPOIlist:
            shopPOIlist.append(item)
            
    
for turl in tt_all:
    for item in getPOIsfromURL(turl):
        if item not in tourPOIlist:
            tourPOIlist.append(item)
    
for eurl in et_all:
    for item in getPOIsfromURL(eurl):
        if item not in eatPOIlist:
            eatPOIlist.append(item)
    
print(len(shopPOIlist), len(eatPOIlist), len(tourPOIlist))

16 1681 717


In [50]:
for shop in shopPOIlist:
    print(shop)

for eat in eatPOIlist:
    print(eat)

for tour in tourPOIlist:
    print(tour)

{'id': 'lj0002989', 'name': '東武百貨店 池袋店', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0002989/', 'locn': '池袋', 'type': '百貨店・デパート'}
{'id': 'lj0003254', 'name': '東急百貨店東横店', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003254/', 'locn': '渋谷', 'type': '百貨店・デパート'}
{'id': 'lj0003209', 'name': '小田急百貨店 新宿店', 'href': 'https://livejapan.com/ja/in-shinjuku/spot-lj0003209/', 'locn': '新宿', 'type': '百貨店・デパート'}
{'id': 'lj0005375', 'name': '日本橋高島屋', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0005375/', 'locn': '東京駅', 'type': '百貨店・デパート'}
{'id': 'lj0005642', 'name': '西武渋谷店', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0005642/', 'locn': '渋谷', 'type': '百貨店・デパート'}
{'id': 'lj0005640', 'name': '西武池袋本店', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0005640/', 'locn': '池袋', 'type': '百貨店・デパート'}
{'id': 'lj0003256', 'name': '東急百貨店本店', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003256/', 'locn': '渋谷', 'type': '百貨店・デパート'}
{'id': 'lj0004986', 'name': 

{'id': 'lj0006234', 'name': 'AK＋', 'href': 'https://livejapan.com/ja/in-jiyugaoka/spot-lj0006234/', 'locn': '自由が丘', 'type': 'その他 カフェ・喫茶店・スイーツ'}
{'id': 'lj0005124', 'name': 'ごはんカフェ MARE（マレ）', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0005124/', 'locn': 'MARE（マレ）', 'type': '池袋 その他 カフェ・喫茶店・スイーツ 言語スタッフ対応：英語 利用可能クレジットカード：VISA Wi-Fi：無料Wi-Fiあり'}
{'id': 'lj0005420', 'name': 'usubane', 'href': 'https://livejapan.com/ja/in-jiyugaoka/spot-lj0005420/', 'locn': '自由が丘', 'type': 'カレー'}
{'id': 'lj0005120', 'name': '楽釜製麺所 新宿西口直売店', 'href': 'https://livejapan.com/ja/in-shinjuku/spot-lj0005120/', 'locn': '新宿西口直売店', 'type': '新宿 うどん・そば'}
{'id': 'lj0005911', 'name': '麺処 直久', 'href': 'https://livejapan.com/ja/in-akihabara/spot-lj0005911/', 'locn': '直久', 'type': '秋葉原 ラーメン'}
{'id': 'lj0006000', 'name': '鶏福', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0006000/', 'locn': '渋谷', 'type': 'ラーメン 言語メニュー対応：英語, 中国語（簡体字）, 中国語（繁体字）, 韓国語 Wi-Fi：無料Wi-Fiあり'}
{'id': 'lj0006002', 'name': '瓦そば専門店 瓦ぐらむ', 'hre